<a href="https://colab.research.google.com/github/njanwani/miniproject1/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def asdf():
  print('asdf')

In [ ]:
def bruh():
  print('bruh')

In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv, json, torch
from scipy import stats
from sklearn.metrics import fbeta_score
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from IPython.display import HTML, Image
from matplotlib import animation, rc
import seaborn as sns

import urllib.request
urllib.request.urlretrieve('https://raw.githubusercontent.com/njanwani/miniproject1/main/train.json', 'train.json')
urllib.request.urlretrieve('https://raw.githubusercontent.com/njanwani/miniproject1/main/test.json', 'test.json')
urllib.request.urlretrieve('https://raw.githubusercontent.com/njanwani/miniproject1/main/tutorial_features.csv', 'tutorial_features.csv')

('tutorial_features.csv', <http.client.HTTPMessage at 0x7f8ce94b2040>)

# Helper functions

In [2]:
def mean_step_speed(coords):
    """Mean step speed of the entire track.
    
    The average per-step speed. Basically the average of distances between points adjacent in time.
    
    Returns
    -------
    float
        The average step speed.
    """

    speeds = []

    for i in range(1, coords.shape[0]):
        # Previous coordinate location
        prev = coords[i-1, 1:]
        # Current coordinate location
        curr = coords[i, 1:]
        
        # Speed in pixels per frame
        curr_speed = np.linalg.norm(curr - prev)
        
        # Accumulate per-step speeds into a list
        speeds.append(curr_speed)
    
    # Return the average of the speeds
    if len(speeds) > 0: 
      return np.mean(speeds)
    else:
      return 0


def stddev_step_speed(coords):
    """Standard deviation of the step speed of the entire track.
    
    The standard deviation of the per-step speed.
    
    Returns
    -------
    float
        The stddev of the step speed.
    """

    speeds = []

    for i in range(1, coords.shape[0]):
        # Previous coordinate location
        prev = coords[i-1, 1:]
        # Current coordinate location
        curr = coords[i, 1:]
        
        # Speed in pixels per frame
        curr_speed = np.linalg.norm(curr - prev)
        
        # Accumulate per-step speeds into a list
        speeds.append(curr_speed)
    
    # Return the standard deviation of the speeds
    if len(speeds) > 0: 
      return np.std(speeds)
    else:
      return 0


def track_length(coords):
    """Standard deviation of the step speed of the entire track.
    
    The standard deviation of the per-step speed.
    
    Returns
    -------
    float
        The length of the entire track.
    """

    lengths = []

    for i in range(1, coords.shape[0]):
        # Previous coordinate location
        prev = coords[i-1,1:]
        # Current coordinate location
        curr = coords[i,1:]
        
        # Speed in pixels per frame
        step_length = np.linalg.norm(curr - prev)
        
        # Accumulate per-step speeds into a list
        lengths.append(step_length)
    
    # Return the sum of the lengths
    return np.sum(lengths)


def e2e_distance(coords):
    """End-to-end distance of the track.
    
    The distance from the start and the end of the given track.
    
    Returns
    -------
    float
        The end-to-end distance of the entire track.
    """
    
    # Start and end of the track
    start = coords[0, 1:]
    end = coords[-1, 1:]
    
    # Return the distance
    return np.linalg.norm(end-start)


def duration(coords):
    """Duration of the track.
    
    The time duration of the track.
    
    Returns
    -------
    int
        The end-to-end duration of the entire track.
    """
    
    # Start and end times of the track
    start_t = coords[0, 0]
    end_t = coords[-1, 0]
    
    # Return the difference
    return end_t - start_t

def min_step(coords):
  speeds = []

  for i in range(1, coords.shape[0]):
      # Previous coordinate location
      prev = coords[i-1, 1:]
      # Current coordinate location
      curr = coords[i, 1:]
      
      # Speed in pixels per frame
      curr_speed = np.linalg.norm(curr - prev)
      
      # Accumulate per-step speeds into a list
      speeds.append(curr_speed)
  
  # Return the average of the speeds
  try:
    return np.min(speeds)
  except:
    return 0


def max_step(coords):
  speeds = []

  for i in range(1, coords.shape[0]):
      # Previous coordinate location
      prev = coords[i-1, 1:]
      # Current coordinate location
      curr = coords[i, 1:]
      
      # Speed in pixels per frame
      curr_speed = np.linalg.norm(curr - prev)
      
      # Accumulate per-step speeds into a list
      speeds.append(curr_speed)
  
  # Return the average of the speeds
  try:
    return np.max(speeds)
  except:
    return 0

def predictability(coords):
  if len(coords) >= 4:
    return stats.linregress(coords[:,0], coords[:,1])[2] # r^2 value
  else:
    return 0

# Pre-processing

In [6]:
FEATURE_LIST = [mean_step_speed, stddev_step_speed, track_length, e2e_distance, duration, min_step, max_step, predictability]

In [7]:
def process(json_file, OUTPUT_FILENAME):
  data = []
  with open(json_file, 'r') as f:
    track_data = json.load(f)

  # Generate the feature csv
  header = ['uid', 'label']
  for featfunc in FEATURE_LIST:
      header.append(featfunc.__name__)

  features = []

  track_uids = track_data.keys()
  for uid in tqdm(track_uids):
      curr_row = {
          'uid': uid,
          'label': track_data[uid]['label']
      }
      
      for featfunc in FEATURE_LIST:
          curr_row[featfunc.__name__] = featfunc(np.array(track_data[uid]['txy']))
      
      features.append(curr_row)

  with open(OUTPUT_FILENAME, 'w') as f:
      writer = csv.DictWriter(f, fieldnames = header)
      writer.writeheader()
      for r in features:
          writer.writerow(r)

  print("Written to:", OUTPUT_FILENAME)

In [4]:
process('train.json', 'train_features.csv')

100%|██████████| 16080/16080 [02:24<00:00, 111.00it/s]


Written to: train_features.csv


In [5]:
process('test.json', 'test_features.csv')

100%|██████████| 477/477 [00:04<00:00, 108.98it/s]

Written to: test_features.csv


# Data Visualizations